In [1]:
import pyconfig
"""
  MaxText/configs/base.yml \
  tokenizer_path=${TOKENIZER_PATH} \
  max_prefill_predict_length=${MAX_PREFILL_PREDICT_LENGTH} \
  max_target_length=${MAX_TARGET_LENGTH} \
  model_name=${MODEL_NAME} \
  ici_fsdp_parallelism=${ICI_FSDP_PARALLELISM} \
  ici_autoregressive_parallelism=${ICI_AUTOREGRESSIVE_PARALLELISM} \
  ici_tensor_parallelism=${ICI_TENSOR_PARALLELISM} \
  scan_layers=${SCAN_LAYERS} \
  weight_dtype=${WEIGHT_DTYPE} \
  dtype=${DTYPE} \
  attention=dot_product \
  per_device_batch_size=${PER_DEVICE_BATCH_SIZE} megablox=${MEGABLOX} quantization=${QUANTIZATION}
"""

pyconfig.initialize(
    [None, "configs/base.yml"],
    run_name="moe_test",
    tokenizer_path="assets/tokenizer.mistral-v3",
    max_prefill_predict_length=1024,
    max_target_length=2048,
    ici_fsdp_parallelism=1,
    ici_autoregressive_parallelism=1,
    ici_tensor_parallelism=4,
    scan_layers=False,
    weight_dtype="bfloat16",
    enable_checkpointing=False,
    model_name="mixtral-8x7b",
    dtype="bfloat16",
    attention="dot_product",
    megablox=True,  # won't be an issue without megablox
    quantization="int8",
    per_device_batch_size=1,
)
config = pyconfig.config

2024-11-16 02:58:47.095580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731725927.108842  284214 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731725927.112809  284214 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Updating keys from env and command line: ['run_name', 'model_name', 'enable_checkpointing', 'dtype', 'quantization', 'weight_dtype', 'megablox', 'scan_layers', 'attention', 'ici_fsdp_parallelism', 'ici_tensor_parallelism', 'ici_autoregressive_parallelism', 'tokenizer_path', 'per_device_batch_size', 'max_target_length', 'max_prefill_predict_length']
Running Model: mixtral-8x7b
Updating following parameters in config

base_emb_dim: 4096
base_num_query_heads: 32
base_num_kv_heads: 8
base_mlp_dim: 14336
base_num_decoder_layers: 1
head_dim: 128
mlp_activations: ['silu', 'linear']
vocab_size: 32000
enable_dropout: False
logits_via_embedding: False
normalization_layer_epsilon: 1e-05
num_experts: 8
num_experts_per_tok: 2
rope_max_timescale: 1000000
decoder_block: mistral
Updating keys from model: ['base_emb_dim', 'base_num_query_heads', 'base_num_kv_heads', 'base_mlp_dim', 'base_num_decoder_layers', 'head_dim', 'mlp_activations', 'vocab_size', 'enable_dropout', 'logits_via_embedding', 'normali

In [2]:
from layers import models, quantizations
import max_utils
import jax

devices_array = max_utils.create_device_mesh(config)
quant = quantizations.configure_quantization(config)
mesh = jax.sharding.Mesh(devices_array, config.mesh_axes)
model = models.Transformer(config, mesh=mesh, quant=quant)

Num_devices: 4, shape (1, 1, 1, 1, 1, 4, 1, 1)


In [3]:
rng = jax.random.PRNGKey(0)

In [4]:
rng1, rng2, rng3 = jax.random.split(rng, 3)
state, state_mesh_annotations = max_utils.setup_decode_state(model, config, rng1, mesh, None)

No decode checkpoint specified - generating random weights.
Running MoE megablox implementation.
No existing checkpoints found, not restoring checkpoint.
Running MoE megablox implementation.


In [5]:
import jaxlib

abstract_params = jax.tree_util.tree_map(
        lambda x: jax.ShapeDtypeStruct(shape=x.shape, dtype=x.dtype, sharding=x.sharding)
        if isinstance(x, jaxlib.xla_extension.ArrayImpl)
        else None,
        state.params,
    )

In [6]:
kv_cache_annotations = max_utils.get_kv_cache_annotations(model, config, rng2, mesh)
kv_cache_shardings = jax.tree_util.tree_map(
        lambda x: jax.sharding.NamedSharding(mesh, x), kv_cache_annotations
    )

Running MoE megablox implementation.


In [7]:
import jax.numpy as jnp
import common_types

@jax.jit
def model_apply(_p, _rng):
  return model.apply(
      _p | {"aqt": {}},
      jnp.ones((1, config.max_prefill_predict_length), dtype=jnp.int32),
      jnp.ones((1, config.max_prefill_predict_length), dtype=jnp.int32),
      decoder_segment_ids=jnp.zeros((1, config.max_prefill_predict_length), dtype=jnp.int32),
      enable_dropout=False,
      model_mode=common_types.MODEL_MODE_PREFILL,
      rngs={"params": _rng},
      mutable=True,
  )

In [8]:
_, new_vars = model_apply(state.params, rng)

Running MoE megablox implementation.


TypeError: add got incompatible shapes for broadcasting: (4, 256, 1024), (1, 1024, 4096).